In [19]:
import refinitiv.data as rd
from datetime import datetime, timedelta
import dateutil
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

rd.open_session()

<refinitiv.data.session.Definition object at 0x163ec86d0 {name='workspace'}>

In [20]:
init_notebook_mode(connected=True)

data = pd.DataFrame()


class SingleVesselHelper:
    @classmethod
    def calculate(cls, df):
        """
        Takes a dataframe with columns:
        # DateTime or AIS Date Time - date when transponder data was received
        # Latitude - vessel latitude
        # Longitude - vessel longitude

        DataFrame must be sorted by DateTime column in descending order (latest records first)

        Returns a new dataframe with columns:
        # Dt.Prev, Dt.Next
        # Lat.Prev, Lat.Next
        # Lon.Prev, Lon.Next
        # Dist
        # Hours
        # AvgSpeed
        """

        # Creating differential dataset
        diffs = cls.__make_diffs(df)

        # Calculating distances
        diffs['Distance'] = diffs.apply(
            lambda row: cls.__dist(
                row['Lon.Next'], row['Lat.Next'], row['Lon.Prev'], row['Lat.Prev']),
            axis=1
        )

        # Calculating hours differences
        diffs['DiffHours'] = diffs.apply(
            lambda row: cls.__diff_hours(row['Dt.Next'], row['Dt.Prev']),
            axis=1
        )

        # Calculating average speeds
        diffs['AvgSpeed'] = diffs['Distance']/diffs['DiffHours']

        return diffs

    @classmethod
    def __diff_hours(cls, next_dt, prev_dt):
        delta = pd.to_datetime(next_dt) - pd.to_datetime(prev_dt)
        return delta.days*24 + delta.seconds/3600

    @classmethod
    def __dist(cls, lon1, lat1, lon2, lat2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * \
            math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        r = 6371 / 1.852  # Km -> nautical miles
        return c * r

    @classmethod
    def __make_diffs(cls, df):
        temp = pd.DataFrame(df)
        if not 'DateTime' in temp and not 'AIS Date Time' in temp:
            raise ValueError('No DateTime column in dataset')
        elif 'AIS Date Time' in temp:
            temp['DateTime'] = temp['AIS Date Time']

        if not 'Latitude' in temp:
            raise ValueError('No Latitude column in dataset')
        if not 'Longitude' in temp:
            raise ValueError('No Longitude column in dataset')

        # Dropping all unnecessary columns
        cropped = temp[['DateTime', 'Latitude', 'Longitude']]

        # Creating shifted dataframes - current
        df_next = pd.DataFrame(cropped).iloc[:-1].reset_index()
        df_next.rename(columns={
                       "DateTime": "Dt.Next", "Latitude": "Lat.Next", "Longitude": "Lon.Next"}, inplace=True)
        del df_next['index']

        # And previous
        df_prev = pd.DataFrame(cropped).iloc[1:].reset_index()
        df_prev.rename(columns={
                       "DateTime": "Dt.Prev", "Latitude": "Lat.Prev", "Longitude": "Lon.Prev"}, inplace=True)
        del df_prev['index']

        # And concatenating
        dataframe = pd.concat([df_next, df_prev], axis=1, join='inner')
        return dataframe


class MultipleVesselHelper:
    def load(self, vessel_ids, days, start_date):
        self.data = rd.get_data(
            vessel_ids,
            ['TR.AssetLocationLatitude',
                'TR.AssetLocationLongitude', 'TR.AssetDateTime'],
            {"VDT": "True", "SDate": days, "EDate": start_date,
                "CH": "Fd", "RH": "IN"}  # "0D", "2018-01-01"
        )
        self.data['Instrument'] = self.data['Instrument'].astype(str)
        self.data['DateTime'] = self.data['AIS Date Time']
        self.__recalculate(vessel_ids)

    def __recalculate(self, vessel_ids):
        self.joined = {}
        for vessel_id in vessel_ids:
            df = self.data[self.data['Instrument'] == vessel_id]
            if df.empty:
                continue
            calculated = SingleVesselHelper.calculate(df)
            self.joined[vessel_id] = calculated

    def __get_map_points(self, vessel_id):
        joined = self.joined[vessel_id]
        data = self.data[self.data['Instrument'] == vessel_id]

        def get_labels():
            def make_label(row):
                if row is None:
                    return "Start"
                else:
                    vel = row['AvgSpeed'] if not math.isnan(
                        row['AvgSpeed']) else 0
                    dist = row['Distance'] if not math.isnan(
                        row['Distance']) else 0
                    return "{:%b, %d '%y} / {:2.2f}nm / {:2.2f}kn".format(pd.to_datetime(row['Dt.Next']), dist, vel)

            def handle_row(row):
                item = joined[joined['Dt.Next'] == row['DateTime']]
                if item.empty:
                    return None
                else:
                    return make_label(item.iloc[0])

            return data.apply(handle_row, axis=1)

        def mark_first_and_last(count, first, last, others):
            array = [others] * count
            array[0] = first
            array[-1] = last
            return array

        return go.Scattergeo(
            lon=data['Longitude'],
            lat=data['Latitude'],
            hovertext=get_labels(),
            name=vessel_id,
            mode='lines+markers',
            marker=dict(
                size=mark_first_and_last(len(data), 15, 15, 6),
                symbol=mark_first_and_last(len(data), 'star', 'star', 0),
                line=dict(width=1)
            ),
            line=go.scattergeo.Line(width=1)
        )

    def plot(self, scope='asia'):
        points = []

        any_data = False

        for vessel_id in self.joined.keys():
            points.append(self.__get_map_points(vessel_id))
            any_data = True

        layout = dict(
            width=900,
            height=600,
            geo=dict(
                scope=scope,
                projection=dict(type='mercator'),
                lonaxis=dict(showgrid=True),
                lataxis=dict(showgrid=True)
            ),
            margin=dict(
                l=0,
                r=0,
                t=0,
                b=0
            ),
            template='plotly_dark',
            colorway=['#6978F7', '#A325E9', '#96E05D', '#4A7FB9',
                      '#E75A2D', '#FBE55A', '#8C8C8D', '#5A54F6'],
            yaxis=dict(gridcolor='black', gridwidth=1, zerolinecolor='black', zerolinewidth=1,
                       side='right', color='#D6D6D5', linecolor='#D6D6D5', ticks='outside'),
            xaxis=dict(gridcolor='black', gridwidth=1, zerolinecolor='black', zerolinewidth=1,
                       color='#D6D6D5', linecolor='#D6D6D5', ticks='outside',
                       rangeslider=dict(visible=True)),
            plot_bgcolor='#1A1A1D',
            paper_bgcolor='#1A1A1D',
        )

        if not any_data:
            print("No data to plot")
            return

        fig = dict(data=points, layout=layout)
        chart = go.Figure(fig)
        chart.update_geos(
            showland=False,
            showocean=True,
            oceancolor='#1A1A1D',
            lonaxis_gridcolor='black',
            lataxis_gridcolor='black'
        )
        chart.show()
        return


def get_scopes():
    return ["world", "usa", "europe", "asia", "africa", "north america", "south america"]


def get_data(imo_list, start_date, geo_scope='asia'):
    v = MultipleVesselHelper()
    v.load(imo_list, '0D', start_date)
    v.plot(geo_scope)
    return v.joined

In [21]:
rd.close_session()